# Importing libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import statistics
import warnings

# Selecting columns

In [2]:
imp_cols=[0, 1, 2, 3, 4, 19]

In [3]:
print("Enter number for optional information in single line with space:")
print("1.on_thyroxine\n2.query_on_thyroxine\n3.on_antithyroid_medication\n4.sick")
print("5.pregnant\n6.thyroid_surgery\n7.I131_treatment\n8.query_hypothyroid")
print("9.query_hyperthyroid\n10.lithium\n11.goitre\n12.tumor\n13.hypopituitary\n14.psych")
print("\n\nIf none enter 0.")
op_cols = list(map(int,input().split()))
op_cols = list(map(lambda x: x + 4, op_cols))
if 4 in op_cols:
  op_cols = []

Enter number for optional information in single line with space:
1.on_thyroxine
2.query_on_thyroxine
3.on_antithyroid_medication
4.sick
5.pregnant
6.thyroid_surgery
7.I131_treatment
8.query_hypothyroid
9.query_hyperthyroid
10.lithium
11.goitre
12.tumor
13.hypopituitary
14.psych


If none enter 0.
2 3


In [4]:
my_dict = {0 : 'age',1: 'sex',2 : 'TSH' ,3 : 'T3',4 : 'TT4',5 : 'on_thyroxine',6 : 'query_on_thyroxine',7 : 'on_antithyroid_medication',8 : 'sick',9 : 'pregnant',
        10 : 'thyroid_surgery',11 : 'I131_treatment',12 : 'query_hypothyroid',13 : 'query_hyperthyroid',
        14 : 'lithium',15 : 'goitre',16 : 'tumor',17 : 'hypopituitary',18 : 'psych',19 : 'Class'}

In [5]:
cols_to_read = imp_cols + op_cols

In [6]:
cols_to_read = list(map(my_dict.get, cols_to_read))

# Importing data

In [7]:
df = pd.read_csv("thyroid_sample.csv", usecols=cols_to_read)
df

,age,sex,TSH,T3,TT4,query_on_thyroxine,on_antithyroid_medication,Class
0,51.734101,1.000000,3.900000,2.471232,157.000000,0.0,0.0,negative
1,51.734101,0.000000,1.900000,1.300000,119.000000,0.0,0.0,negative
2,51.734101,1.000000,3.900000,2.471232,157.000000,0.0,0.0,negative
3,51.734101,0.000000,1.900000,1.300000,119.000000,0.0,0.0,negative
4,51.734101,1.000000,2.700000,2.022137,77.000000,0.0,0.0,negative
...,...,...,...,...,...,...,...,...
26490,43.816279,0.563256,2.597528,1.767767,36.644652,0.0,0.0,secondary hypothyroid
26491,42.760888,0.352178,1.629735,2.042169,31.156620,0.0,0.0,secondary hypothyroid
26492,42.047673,0.209535,0.975716,2.227605,27.447900,0.0,0.0,secondary hypothyroid
26493,41.725528,0.145106,0.680309,2.311363,25.772745,0.0,0.0,secondary hypothyroid


In [8]:
X = df.drop('Class', axis = 1)
y = df['Class']

In [9]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [10]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Model training


In [11]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(learning_rate=0.1, max_depth=4, n_estimators=217)
xgb_model.fit(X_scaled, y_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=217, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance')
knn_model.fit(X_scaled, y_encoded)

KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance')

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(max_depth=12, max_features='sqrt', n_estimators=211)
rf_model.fit(X_scaled, y_encoded)

RandomForestClassifier(max_depth=12, n_estimators=211)

# Model Testing

In [15]:
age = float(input("Enter age: "))
sex = input("Enter gender M/F: ")
sex = 1 if sex == 'F' else 0
tsh = float(input("Enter TSH value: "))
t3 = float(input("Enter T3 value: "))
tt4 = float(input("Enter TT4 value: "))
must_param = [age, sex, tsh, t3, tt4]
op_param=[]
if (op_cols!=[]):
  i = 0
  for i in range(len(op_cols)):
    print("Enter",my_dict[op_cols[i]],"value:",end=' ')
    temp = input()
    if temp == 't':
      temp = 1
    else: temp = 0
    op_param.append(temp)
user_input = np.array([must_param + op_param])
scaled_minmax = scaler.transform(user_input)
warnings.filterwarnings('ignore', message="X does not have valid feature names, but MinMaxScaler*")
# Creating a Dataframe with the scaled user input
user_df = pd.DataFrame(scaled_minmax)

predicted_xgb = xgb_model.predict(user_df)
predicted_knn = knn_model.predict(user_df)
predicted_rf = rf_model.predict(user_df)
predictions = [predicted_xgb[0], predicted_knn[0], predicted_rf[0]]

predicted_result_encoded = np.array([statistics.mode(predictions)])
predicted_class = label_encoder.inverse_transform(predicted_result_encoded)

print(f"The predicted result is: {predicted_class[0]}")
warnings.filterwarnings('ignore')

Enter age: 27
Enter gender M/F: F
Enter TSH value: 56
Enter T3 value: 2
Enter TT4 value: 36
Enter query_on_thyroxine value: F
Enter on_antithyroid_medication value: T
The predicted result is: primary hypothyroid
